# Generate data with only checkbox region

define parameters

In [7]:
data_path = '../../../Documents/obj_samples/Linz/val/val_data'
label_path = '../../../Documents/obj_samples/Linz/val/val_label'
save_path = '../../../Documents/obj_samples/Linz/cropped_train_data_2'
template_path = '../../../Documents/obj_samples/Linz/template_p2.png'
cb_region_y = 40
cb_region_x = 1084
cb_region_w = 150
cb_region_h = 1110

* grab all image and label path as lists
* create saving folder

In [8]:
from pathlib import Path
import cv2 as cv
from os import makedirs

image_path_list = list(Path(data_path).glob('*G'))
label_path_list = list(Path(label_path).glob('*t'))
makedirs(save_path, exist_ok=True)

## Unwarp image, select checkbox region, change label coordinate accordingly
* load image one by one
* load according labels
* do unwarpping
* crop cb region
* map label coordinate to new space
* save to path

In [9]:
def map_feature(image, template, ratio_thresh):
    akaze = cv.AKAZE_create()
    key_points, descriptors = akaze.detectAndCompute(template, None)
    key_points2, descriptors2 = akaze.detectAndCompute(image, None)
    matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_BRUTEFORCE_HAMMING)
    knn_matches = matcher.knnMatch(descriptors, descriptors2, 2)
    good_matches = [m for m, n in knn_matches if m.distance < ratio_thresh * n.distance]
    img_matches = np.empty((max(template.shape[0], image.shape[0]), template.shape[1]+image.shape[1], 3), dtype=np.uint8)
    cv.drawMatches(template, key_points, image, key_points2, good_matches, img_matches, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    obj = np.empty((len(good_matches), 2), dtype=np.float32)
    scene = np.empty((len(good_matches), 2), dtype=np.float32)
    for i in range(len(good_matches)):
        obj[i, 0] = key_points[good_matches[i].queryIdx].pt[0]
        obj[i, 1] = key_points[good_matches[i].queryIdx].pt[1]
        scene[i, 0] = key_points2[good_matches[i].trainIdx].pt[0]
        scene[i, 1] = key_points2[good_matches[i].trainIdx].pt[1]
    H, _ = cv.findHomography(scene, obj, cv.RANSAC)
    return cv.warpPerspective(src=image, M=H, dsize=template.shape[1::-1]), H

def warpPerspective_xy(H, x, y):
    return (H[0,0]*x + H[0,1]*y + H[0,2]) / (H[2,0]*x + H[2,1]*y + H[2,2]), (H[1,0]*x + H[1,1]*y + H[1,2]) / (H[2,0]*x + H[2,1]*y + H[2,2])

def map_coordinate_to_unwarpped_cb_region(x_min, y_min, x_max, y_max):
    new_x_min, new_y_min = warpPerspective_xy(H, x_min, y_min)
    new_x_max, new_y_max = warpPerspective_xy(H, x_max, y_max)
    return max(new_x_min-cb_region_x, 0),  max(new_y_min-cb_region_y, 0), min(new_x_max-cb_region_x, cb_region_w), min(new_y_max-cb_region_y, cb_region_h)

In [10]:
import csv
import numpy as np
template = cv.imread(template_path)
for label_path in label_path_list:
    image_path = Path(data_path).joinpath(label_path.name[:-4] + '.JPG')
    image = cv.imread(str(image_path))
    unwarpped_image, H = map_feature(image, template, .75)
    with open(label_path) as fl:
        lines = fl.readlines() 
        annotation_list = []
        for line in lines:
            obj = line.strip().split(' ')
            x_min, y_min, x_max, y_max = map_coordinate_to_unwarpped_cb_region(*[float(coordinate) for coordinate in obj[4:8]])
            annotation_list.append(['checkbox', 0, 0, 0, x_min, y_min, x_max, y_max, 0, 0, 0, 0, 0, 0, 0])
    cropped = unwarpped_image[cb_region_y:cb_region_y+cb_region_h, cb_region_x:cb_region_x+cb_region_w, :]
    cv.imwrite(str(Path(save_path).joinpath(image_path.name)), cropped)
    with open(Path(save_path).joinpath(label_path.name), 'w') as fl:
        csv.writer(fl,delimiter=" ").writerows(annotation_list)

## data augmentation

In [25]:
import csv
from os import makedirs
from pathlib import Path, PurePosixPath
from typing import List

import imageio
import imgaug as ia
from absl import app, flags
from cached_property import cached_property
from imgaug import augmenters
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from nptyping import Array


ia.seed(5)

class ImageAug():
    def __init__(self, base_path: str, augment_time: int, image_format: str):
        self._save_path = Path(base_path).parent.joinpath('augmented')
        base_path = Path(base_path)
        self._image_path_list = list(base_path.glob('*{}'.format(image_format)))
        self._label_path_list = list(base_path.glob('*txt'))
        self._augment_time = augment_time
        self._image_format = image_format
        makedirs(str(self._save_path), exist_ok=True)

    def augment_image_label(self):
        for image_path in self._image_path_list:
            image_name = image_path.name[:-4]
            label_path = [label_path for label_path in self._label_path_list if image_name in str(label_path)]
            assert len(label_path) is 1, 'Found more than one label file for image {}'.format(image_name)
            for idx in range(self._augment_time):
                image, label = self._get_image_label(image_path, label_path[0])
                aug = self._aug_seq(image=image, bounding_boxes=label)
                self._save_image_label(*aug, str(self._save_path)+'/{}_aug_{}'.format(image_name, str(idx)))

    @cached_property
    def _aug_seq(self):
        return augmenters.Sequential([
            augmenters.Sometimes(1, augmenters.Multiply((0.9, 1.1), per_channel=0.5)),
            augmenters.Sometimes(1, augmenters.MotionBlur(k=15)),
            augmenters.Affine(translate_percent={"x": (-0.02, 0.02), "y": (-0.02, 0.02)}, scale=(0.98, 1.02))
                ], random_order=True)

    def _get_image_label(self, image_path: str, label_path: str) -> [Array, List[BoundingBox]]:
        image = imageio.imread(image_path)
        with open(label_path) as f:
            lines = f.readlines()
        label_list_raw = [line.strip().split(' ') for line in lines]
        coordinate_list = [list(map(float, label[4:8])) for label in label_list_raw]
        class_list = [label[0] for label in label_list_raw]
        boundingbox_list = [BoundingBox(*label[1], label=label[0]) for label in zip(class_list, coordinate_list)]
        label = BoundingBoxesOnImage(boundingbox_list, shape=image.shape)
        return image, label

    def _save_image_label(self, image: Array, label: BoundingBoxesOnImage, save_path: str):
        imageio.imwrite(save_path+'.JPG', image)
        annotation = [[bbs.label, 0, 0, 0, bbs.x1, bbs.y1, bbs.x2, bbs.y2, 0, 0, 0, 0, 0, 0, 0] for bbs in label.bounding_boxes]
        with open(save_path+'.txt', 'w') as fl:
            csv.writer(fl,delimiter=" ").writerows(annotation)


In [26]:
base_dir = '../../../Documents/obj_samples/Linz/cropped_train_data'
image_aug = ImageAug(base_dir, 5, 'JPG')
image_aug.augment_image_label()

make sure all augmented cb coordinates are larger than zero and within the boundary

In [ ]:
save_path = Path(aug_save_dir).parent.joinpath('augmented')
aug_label_path_list = save_path.glob('*txt')
for aug_label_path in aug_label_path_list:
    with open(label_path) as f:
        lines = f.readlines()
    coordinate_list = [line.strip().split(' ') for line in lines]
    label_coordinate = [max(float(number), 0) for coordinate in coordinate_list for number in coordinate[4:8]]
    coordinate_list = [[*label[:4], *label_coordinate[0:2], min(label_coordinate[2], cb_region_w), min(label_coordinate, cb_region_h), *label[8:]] for label in coordinate_list]

## Visulization

In [29]:
image_dir = '../../../Documents/obj_samples/Linz/cropped_train_data'
image_path_list = list(Path(image_dir).glob('*G'))

In [49]:
import cv2 as cv
max_h=1110
max_w=150
for image_path in image_path_list:
    image = cv.imread(str(image_path))
    label_path = image_path.parent.joinpath(image_path.name[:-4] + '.txt')
    with open(label_path) as f:
        lines = f.readlines()
    coordinate_list = [line.strip().split(' ') for line in lines]
    for coordinate in coordinate_list:
        x_min, y_min, x_max, y_max = [int(float(number)) for number in coordinate[4:8]]
        image = cv.rectangle(img=image, pt1=(x_min, y_min), pt2=(x_max, y_max), color=(0, 0, 255), thickness=2)
    image_save_path = Path('../../../Documents/obj_samples/Linz/cropped_train_data_viz').joinpath(image_path.name)
    cv.imwrite(str(image_save_path), image)

## Change label

In [39]:
def save_label(label_path, label_list):
    with open(label_path, 'w') as fl:
        csv.writer(fl,delimiter=" ").writerows(label_list)

In [ ]:
offset_x, offset_y = 1090, 40
label_path = '../../../Documents/obj_samples/Linz/cropped_train_data/IMG_1526.txt'
image_path = Path('../../../Documents/obj_samples/Linz/cropped_train_data').joinpath(Path(label_path).name[:-4]+'.JPG')
image = cv.imread(str(image_path))
with open(label_path) as f:
    lines = f.readlines()
coordinate_list = [line.strip().split(' ') for line in lines]
coordinate_list = [[*label[:4],\
                    max(float(label[4])-offset_x, 0),\
                    max(float(label[5])-offset_y, 0),\
                    min(float(label[6])-offset_x, cb_region_w),\
                    min(float(label[7])-offset_y, cb_region_h),\
                    *label[8:]] for label in coordinate_list]
for coordinate in coordinate_list:
    image = cv.rectangle(img=image, pt1=(int(coordinate[4]), int(coordinate[5])), pt2=(int(coordinate[6]), int(coordinate[7])), color=(0, 0, 255), thickness=2)
cv.imwrite(str(Path('../../../Documents/obj_samples/Linz/cropped_train_data_viz').joinpath(Path(label_path).name[:-4]+'.JPG')), image)
#save_label(label_path, coordinate_list)